In [1]:
import pandas as pd


In [2]:
df = pd.read_csv('data/gemstone.csv')

In [3]:
df = df.drop(labels = ["id"],axis = 1)
df

,carat,cut,color,clarity,depth,table,x,y,z,price
0,1.52,Premium,F,VS2,62.2,58.0,7.27,7.33,4.55,13619
1,2.03,Very Good,J,SI2,62.0,58.0,8.06,8.12,5.05,13387
2,0.70,Ideal,G,VS1,61.2,57.0,5.69,5.73,3.50,2772
3,0.32,Ideal,G,VS1,61.6,56.0,4.38,4.41,2.71,666
4,1.70,Premium,G,VS2,62.6,59.0,7.65,7.61,4.77,14453
...,...,...,...,...,...,...,...,...,...,...
193568,0.31,Ideal,D,VVS2,61.1,56.0,4.35,4.39,2.67,1130
193569,0.70,Premium,G,VVS2,60.3,58.0,5.75,5.77,3.47,2874
193570,0.73,Very Good,F,SI1,63.1,57.0,5.72,5.75,3.62,3036
193571,0.34,Very Good,D,SI1,62.9,55.0,4.45,4.49,2.81,681


In [4]:
# independant and dependat features
X = df.drop(labels= ["price"], axis= 1)
y = df[['price']]
y


,price
0,13619
1,13387
2,2772
3,666
4,14453
...,...
193568,1130
193569,2874
193570,3036
193571,681


In [5]:
X

,carat,cut,color,clarity,depth,table,x,y,z
0,1.52,Premium,F,VS2,62.2,58.0,7.27,7.33,4.55
1,2.03,Very Good,J,SI2,62.0,58.0,8.06,8.12,5.05
2,0.70,Ideal,G,VS1,61.2,57.0,5.69,5.73,3.50
3,0.32,Ideal,G,VS1,61.6,56.0,4.38,4.41,2.71
4,1.70,Premium,G,VS2,62.6,59.0,7.65,7.61,4.77
...,...,...,...,...,...,...,...,...,...
193568,0.31,Ideal,D,VVS2,61.1,56.0,4.35,4.39,2.67
193569,0.70,Premium,G,VVS2,60.3,58.0,5.75,5.77,3.47
193570,0.73,Very Good,F,SI1,63.1,57.0,5.72,5.75,3.62
193571,0.34,Very Good,D,SI1,62.9,55.0,4.45,4.49,2.81


In [23]:
categorical_col = X.select_dtypes(include = 'object').columns
categorical_col

Index(['cut', 'color', 'clarity'], dtype='object')

In [24]:
numerical_col = X.select_dtypes(exclude = 'object').columns
numerical_col

Index(['carat', 'depth', 'table', 'x', 'y', 'z'], dtype='object')

In [25]:
df[categorical_col].isnull().sum()

cut        0
color      0
clarity    0
dtype: int64

In [26]:
df.isnull().sum()

carat      0
cut        0
color      0
clarity    0
depth      0
table      0
x          0
y          0
z          0
price      0
dtype: int64

#### simple imputer deals with a missing values in the dataset by replacing 
#### simple stragies like mean, mode and median
#### If we have an outlier then obviously use mean or median
#### If have an categorical data use most frequent(mode)
#### where our categorical feature have rank we use ordinal encoding 
#### othere wise we use onehot encoder

In [10]:
# Define the custom ranking for each ordinal variable
cut_categories = ['Fair', 'Good', 'Very Good','Premium','Ideal']
color_categories = ['D', 'E', 'F', 'G', 'H', 'I', 'J']
clarity_categories = ['I1','SI2','SI1','VS2','VS1','VVS2','VVS1','IF']

In [27]:
from sklearn.impute import SimpleImputer # handling the missing value
from sklearn.preprocessing import StandardScaler # handle feature scaling (handle high relation between veriable)
from sklearn.preprocessing import OrdinalEncoder # Ordinal Encoding 
# pipelines = This is use for combining all the upper steps
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer # this is for grouping all the upper thing

In [28]:
# Imputing is also done for new training data for our training model if a new test data is having 
# missing value then we need to handle it train our model

In [29]:
## Numerical pipline
## pipeline is use for automate the EDA technique which we did in EDA.ipynb

numerical_pipeline  = Pipeline(
    steps= [
        # here imputer is the veriable or object of SimpleImputer method
        ('imputer', SimpleImputer(strategy= 'median')), # handle missing value
        ('scaler', StandardScaler()) # handle scaling 
    ]
)

## Encoding for categorcial feature
categorical_pipline = Pipeline(
    steps= [
        ('imputer', SimpleImputer(strategy= 'most_frequent')),
        ('ordinal_encoder', OrdinalEncoder(categories= [ cut_categories, color_categories, clarity_categories])),
        ('scaler', StandardScaler())

    ]
    )
# combine numerical_pipeline with categorical_pipeline by using column_transformer
# On numerical column we are applying numerical pipeline
pre_processor = ColumnTransformer([
    ('num_pipeline', numerical_pipeline, numerical_col),
    ('cat_pipeline', categorical_pipline, categorical_col)
])

In [30]:
# train test split
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size= 0.30, random_state= 30
)

In [31]:

pre_processor

ColumnTransformer(transformers=[('num_pipeline',
                                 Pipeline(steps=[('imputer',
                                                  SimpleImputer(strategy='median')),
                                                 ('scaler', StandardScaler())]),
                                 Index(['carat', 'depth', 'table', 'x', 'y', 'z'], dtype='object')),
                                ('cat_pipeline',
                                 Pipeline(steps=[('imputer',
                                                  SimpleImputer(strategy='most_frequent')),
                                                 ('ordinal_encoder',
                                                  OrdinalEncoder(categories=[['Fair',
                                                                              'Good',
                                                                              'Very '
                                                                              'Good',
                                                                              'Premium',
                                                                              'Ideal'],
                                                                             ['D',
                                                                              'E',
                                                                              'F',
                                                                              'G',
                                                                              'H',
                                                                              'I',
                                                                              'J'],
                                                                             ['I1',
                                                                              'SI2',
                                                                              'SI1',
                                                                              'VS2',
                                                                              'VS1',
                                                                              'VVS2',
                                                                              'VVS1',
                                                                              'IF']])),
                                                 ('scaler', StandardScaler())]),
                                 Index(['cut', 'color', 'clarity'], dtype='object'))])

In [32]:
# Feature scalling
# this is having standard normalized data with mue = 0 and sigma = 1 also applied z_score to X_train data
X_train = pd.DataFrame(pre_processor.fit_transform(X_train), columns= pre_processor.get_feature_names_out())
X_train

,num_pipeline__carat,num_pipeline__depth,num_pipeline__table,num_pipeline__x,num_pipeline__y,num_pipeline__z,cat_pipeline__cut,cat_pipeline__color,cat_pipeline__clarity
0,-0.975439,-0.849607,-0.121531,-1.042757,-1.080970,-1.123150,0.874076,1.528722,1.352731
1,0.235195,1.833637,-0.121531,0.318447,0.279859,0.485354,-2.144558,-0.935071,-0.646786
2,0.494617,0.815855,0.399800,0.570855,0.606458,0.673737,-0.132136,0.296826,0.686225
3,-1.018676,0.260701,0.921131,-1.214034,-1.244270,-1.195605,-0.132136,0.296826,0.019720
4,-0.953821,-0.664555,-0.642862,-1.069801,-1.044681,-1.094168,0.874076,2.144670,1.352731
...,...,...,...,...,...,...,...,...,...
135496,-1.040295,-0.016876,-0.642862,-1.268122,-1.244270,-1.239078,0.874076,-0.935071,-0.646786
135497,0.991842,0.168176,-0.642862,1.048629,1.114501,1.079486,0.874076,0.296826,-1.313292
135498,0.451380,1.556060,-0.642862,0.516768,0.588314,0.702719,-2.144558,0.296826,-0.646786
135499,0.667565,-1.774863,1.442462,0.868337,0.951202,0.688228,0.874076,0.296826,0.686225


In [33]:
# here only applied z score without knowing model mue and sigma to avoid data leakage
X_test = pd.DataFrame(pre_processor.transform(X_test), columns= pre_processor.get_feature_names_out())
X_test

,num_pipeline__carat,num_pipeline__depth,num_pipeline__table,num_pipeline__x,num_pipeline__y,num_pipeline__z,cat_pipeline__cut,cat_pipeline__color,cat_pipeline__clarity
0,-0.564688,-0.942132,-0.642862,-0.429765,-0.464061,-0.500036,-0.132136,-0.935071,0.019720
1,-0.175556,1.000906,-0.121531,-0.042137,-0.028595,0.036132,-1.138347,0.912774,-0.646786
2,-1.061913,0.260701,-0.121531,-1.304180,-1.298703,-1.268060,0.874076,0.912774,2.685743
3,0.970223,-0.201927,1.963794,1.048629,0.996563,0.978049,-0.132136,0.296826,0.019720
4,-0.932202,-1.312235,0.399800,-1.006699,-0.990248,-1.065186,-0.132136,-0.935071,0.686225
...,...,...,...,...,...,...,...,...,...
58067,1.013460,1.185958,-0.642862,1.003556,1.041924,1.151941,-1.138347,0.912774,0.019720
58068,-0.997058,0.260701,-1.164193,-1.141917,-1.126331,-1.108659,0.874076,-0.319122,2.019237
58069,-0.197174,-3.347799,1.442462,0.102096,0.071199,-0.224706,-0.132136,2.144670,0.019720
58070,-0.824110,-0.201927,-0.121531,-0.853450,-0.881382,-0.876803,0.874076,0.296826,-0.646786


In [34]:

## Model Training

from sklearn.linear_model import LinearRegression,Lasso,Ridge,ElasticNet
from sklearn.metrics import r2_score,mean_absolute_error,mean_squared_error

In [35]:
regression =  LinearRegression()
regression.fit(X_train, y_train)

LinearRegression()

In [36]:
regression.coef_

array([[ 6433.66003594,  -132.75843566,   -70.42922179, -1720.30971463,
         -499.29302619,   -63.39317848,    72.44537247,  -460.41604642,
          650.76431652]])

In [37]:
regression.intercept_

array([3970.76628955])

In [38]:
import numpy as np
def evaluate_model(true, predicted):
  mae = mean_absolute_error(true, predicted),
  mse = mean_squared_error(true, predicted),
  rmse = np.sqrt(mean_squared_error(true, predicted))
  r2_square = r2_score(true, predicted)
  return mae, mse, rmse,r2_square
 


In [39]:
## Train multiple models

models={
    'LinearRegression':LinearRegression(),
    'Lasso':Lasso(),
    'Ridge':Ridge(),
    'Elasticnet':ElasticNet()
}

trained_model_list = []
model_list = []
r2_list = []
for i in range(len(list(models))):
    model = list(models.values())[i]
    model.fit(X_train, y_train)
    
    # make prediction
    y_pred = model.predict(X_test)

    mae, mse, rmse, r2_square = evaluate_model(y_test, y_pred)
    print(list(models.keys())[i])
    model_list.append(list(models.keys())[i])
    print("Model Training Performance")
    print('mae : ', mae)
    print('mse : ', mse)
    print('rmse : ', rmse)
    print("R2 score",r2_square*100)

    r2_list.append(r2_square)

    print('='*35)
    print('\n')

LinearRegression
Model Training Performance
mae :  (674.0255115796832,)
mse :  (1028002.7598132559,)
rmse :  1013.9047094344004
R2 score 93.68908248567512


Lasso
Model Training Performance
mae :  (675.0716923362158,)
mse :  (1027949.4559693958,)
rmse :  1013.8784226767013
R2 score 93.68940971841704


Ridge
Model Training Performance
mae :  (674.0555800798212,)
mse :  (1028005.2293677665,)
rmse :  1013.9059272771643
R2 score 93.68906732505938


Elasticnet
Model Training Performance
mae :  (1060.7368759154729,)
mse :  (2351365.382289642,)
rmse :  1533.4162456064048
R2 score 85.56494831165182




In [40]:
print(model_list)

['LinearRegression', 'Lasso', 'Ridge', 'Elasticnet']


In [22]:
print(r2_list)

[]
